# Creación de fichero .csv con el histórico de las temperaturas y los tokens

In [7]:
import pandas as pd

# Cargar el archivo CSV original
data = pd.read_csv('predicciones_temperatura.csv')

# Redondear las temperaturas a un decimal
data['Temperatura'] = data['Temperatura'].round(1)

# Crear una lista para almacenar las frases formateadas
frases_formateadas = []

# Procesar cada fila del DataFrame
for index, row in data.iterrows():
    frase = f"<bos> El día <|{row['Fecha']}|Fecha|> en la estación de <|{row['Estacion']}|Estacion|> hará {row['Temperatura']} grados<eos>"
    frases_formateadas.append(frase)

# Crear un nuevo DataFrame con las frases formateadas
df_formateado = pd.DataFrame(frases_formateadas, columns=['Texto'])

# Guardar el nuevo DataFrame como un archivo CSV sin encabezado
df_formateado.to_csv('historico_temperaturas.csv', index=False, header=False)

# Predict día siguiente a hoy

**Leemos fichero .csv obtenido de Azure con las últimas 30 temperaturas para cada estación**

In [10]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv("datos_temperaturas_ultimos_30_dias.csv")

# Crear una matriz de temperaturas por estación
temperaturas_por_estacion = {}

# Iterar sobre cada estación
for estacion, datos_estacion in df.groupby('Estacion'):
    # Obtener las temperaturas de la estación actual
    temperaturas = datos_estacion['Temperatura'].tolist()
    # Agregar las temperaturas a la matriz por estación
    temperaturas_por_estacion[estacion] = temperaturas

# Crear una lista de temperaturas por estación
temperaturas_por_estacion_lista = []

# Iterar sobre cada estación y agregar a la lista
for estacion, temperaturas in temperaturas_por_estacion.items():
    temperaturas_por_estacion_lista.append([estacion, temperaturas])

# Imprimir la lista de temperaturas por estación
print(temperaturas_por_estacion_lista)


[['Avenida Francia', [18.677032, 17.506887, 17.452826, 17.59764, 17.54906, 17.919657, 18.577843, 18.686684, 17.990097, 17.556318, 18.687288, 18.713657, 18.110184, 18.851501, 18.814316, 18.445217, 18.572899, 15.283064, 18.850576, 18.07406, 17.696201, 18.367697, 18.540298, 18.530657, 18.094551, 18.67478, 17.985226, 16.678644, 18.730598, 18.39048]], ['Bulevard Sud', [17.501852, 18.536959, 18.627752, 17.769398, 19.255806, 21.749155, 18.51207, 18.766434, 18.530262, 22.785456, 17.594551, 18.58283, 18.801262, 18.735569, 17.482862, 17.861265, 15.950268, 18.628874, 18.809685, 18.029972, 17.778767, 18.570105, 17.805681, 18.966179, 18.435371, 17.373787, 18.603212, 18.875723, 18.337997, 18.659372]], ['Conselleria Meteo', [18.349537, 18.495623, 18.657303, 18.747286, 15.421509, 18.446932, 17.54641, 17.727663, 22.397213, 17.553139, 18.646265, 18.473814, 18.650372, 18.747307, 18.736557, 18.565292, 17.940903, 17.785965, 18.189781, 18.53672, 21.702312, 18.114817, 17.79291, 17.92919, 18.117544, 18.563349

**Realizamos predict para cada una de las estaciones**

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from tensorflow.keras.models import load_model

window_size = 30
horizon = 1
num_features = 1

model = Sequential([
    SimpleRNN(50, input_shape=(window_size, num_features), return_sequences=False),
    Dense(horizon)
])

model.compile(optimizer='adam', loss='mse')

modelo = load_model('../Backend/models/rnn_temperatura.h5')
temperaturas = []
modelo.predict()

In [23]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# Cargar el modelo entrenado
modelo = load_model('../Backend/models/rnn_temperatura.h5')

# Leer los datos del archivo CSV
datos = pd.read_csv('datos_temperaturas_ultimos_30_dias.csv')

# Crear una variable para almacenar las temperaturas de cada estación
temperaturas_por_estacion = {}

# Agrupar los datos por estación y combinar las temperaturas de cada estación en una lista
for estacion, grupo in datos.groupby('Estacion'):
    temperaturas_por_estacion[estacion] = grupo['Temperatura'].tolist()

# Ahora, temperaturas_por_estacion es un diccionario donde las claves son los nombres de las estaciones
# y los valores son listas que contienen las temperaturas de cada estación

# Realizar la predicción para cada estación
predicciones_por_estacion = {}
for estacion, temperaturas in temperaturas_por_estacion.items():
    # Cambiar la forma de los datos para que sean compatibles con el modelo
    # El modelo espera una entrada de forma (1, window_size, num_features)
    X_prediccion = np.array(temperaturas).reshape((1, window_size, num_features))
    # Realizar la predicción con el modelo
    prediccion = modelo.predict(X_prediccion)
    # Almacenar la predicción en el diccionario
    predicciones_por_estacion[estacion] = prediccion

# Imprimir las predicciones para cada estación con un solo decimal
for estacion, prediccion in predicciones_por_estacion.items():
    # Formatear la predicción para mostrar solo un decimal
    prediccion_formateada = "{:.1f}".format(prediccion[0][0])
    print(f"Predicción de temperatura para la estación {estacion}: {prediccion_formateada}")


1/1 [==============================] - 0s 12ms/step
Predicción de temperatura para la estación Avenida Francia: 18.3
Predicción de temperatura para la estación Bulevard Sud: 18.7
Predicción de temperatura para la estación Conselleria Meteo: 18.3
Predicción de temperatura para la estación Moli del Sol: 14.8
Predicción de temperatura para la estación Nazaret Meteo: 18.6
Predicción de temperatura para la estación Pista Silla: 18.9
Predicción de temperatura para la estación Politecnico: 13.6
Predicción de temperatura para la estación Puerto Moll Tras Ponent: 17.9
Predicción de temperatura para la estación Puerto Valencia: 18.8
Predicción de temperatura para la estación Puerto llit antic Turia: 18.3
Predicción de temperatura para la estación Valencia Centro: 18.5
Predicción de temperatura para la estación Valencia Olivereta: 15.6
Predicción de temperatura para la estación Viveros: 15.4


In [26]:
import csv
from datetime import datetime, timedelta

# Obtener la fecha de mañana en el formato deseado
fecha_mañana = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")

# Nombre del archivo CSV de salida
nombre_archivo_salida = "predicciones_temperaturas_manyana.csv"

# Abrir el archivo CSV en modo de escritura
with open(nombre_archivo_salida, mode='w', newline='') as file:
    # Crear un escritor CSV con un delimitador personalizado y comillas para los campos
    writer = csv.writer(file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    # Escribir las predicciones para cada estación en el archivo CSV
    for estacion, prediccion in predicciones_por_estacion.items():
        # Formatear la predicción para mostrar solo un decimal
        prediccion_formateada = "{:.1f}".format(prediccion[0][0])
        # Crear la cadena con el formato deseado
        fila = f"<bos> El día <|{fecha_mañana}|Fecha|> en la estación de <|{estacion}|Estacion|> hará {prediccion_formateada} grados<eos>"
        # Escribir la fila en el archivo CSV
        writer.writerow([fila])

print(f"Se han guardado las predicciones en el archivo {nombre_archivo_salida}")


Se han guardado las predicciones en el archivo predicciones_temperaturas_manyana.csv


**Cargamos datos al fichero .csv general para volver a hacer FineTuning**

In [28]:
# Nombre del archivo CSV de salida para las predicciones
nombre_archivo_predicciones = "predicciones_temperaturas_manyana.csv"

# Nombre del archivo CSV de historico
nombre_archivo_historico = "historico_temperaturas.csv"

# Leer el contenido de predicciones_temperaturas.csv
with open(nombre_archivo_predicciones, mode='r') as predicciones_file:
    predicciones_content = predicciones_file.read()

# Escribir el contenido al final de historico_temperaturas.csv
with open(nombre_archivo_historico, mode='a') as historico_file:
    historico_file.write(predicciones_content)

print("Se ha añadido el contenido de predicciones_temperaturas.csv al final de historico_temperaturas.csv")


Se ha añadido el contenido de predicciones_temperaturas.csv al final de historico_temperaturas.csv


# Inicio Entrenamiento SOTOM

In [29]:
import os
import time
import datetime

import numpy as np
import random
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AdamW, get_linear_schedule_with_warmup

In [30]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)

if torch.cuda.is_available():
    print("Usar GPU")
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    device = torch.device("cuda")
    batch_size = 3

else:
    print("usar CPU")
    device = torch.device("cpu")
    batch_size = 1


usar CPU


**Leer .csv de fechas**

In [30]:
import os
import csv

# Definir la ruta absoluta del archivo CSV
ruta_absoluta_csv = os.path.abspath('../Backend/Python/ObtenerDatosDataBricks/fechas_distintas.csv')

# Verificar que el archivo existe antes de intentar leerlo
if os.path.exists(ruta_absoluta_csv):
    # Leer el archivo CSV y cargar los datos en una lista
    aditional_special_tokens_array = []
    with open(ruta_absoluta_csv, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            aditional_special_tokens_array.append("<|Fecha|" + row['Fecha'] + "|>")
    print(aditional_special_tokens_array)
else:
    print(f"El archivo {ruta_absoluta_csv} no existe.")


['<|Fecha|2005-01-16|>', '<|Fecha|2022-03-28|>', '<|Fecha|2006-05-17|>', '<|Fecha|2007-04-20|>', '<|Fecha|2019-06-04|>', '<|Fecha|2022-07-31|>', '<|Fecha|2007-11-15|>', '<|Fecha|2007-11-23|>', '<|Fecha|2009-07-25|>', '<|Fecha|2009-11-22|>', '<|Fecha|2020-08-24|>', '<|Fecha|2021-01-27|>', '<|Fecha|2021-06-22|>', '<|Fecha|2021-08-27|>', '<|Fecha|2021-10-11|>', '<|Fecha|2021-11-13|>', '<|Fecha|2021-12-18|>', '<|Fecha|2016-03-01|>', '<|Fecha|2017-08-11|>', '<|Fecha|2017-09-11|>', '<|Fecha|2018-05-28|>', '<|Fecha|2018-08-10|>', '<|Fecha|2019-05-08|>', '<|Fecha|2012-04-17|>', '<|Fecha|2012-10-06|>', '<|Fecha|2013-01-22|>', '<|Fecha|2005-06-06|>', '<|Fecha|2013-03-26|>', '<|Fecha|2013-05-21|>', '<|Fecha|2006-05-21|>', '<|Fecha|2013-09-09|>', '<|Fecha|2014-09-26|>', '<|Fecha|2014-11-12|>', '<|Fecha|2015-03-09|>', '<|Fecha|2010-08-11|>', '<|Fecha|2015-05-19|>', '<|Fecha|2004-02-23|>', '<|Fecha|2007-04-19|>', '<|Fecha|2022-07-27|>', '<|Fecha|2022-08-02|>', '<|Fecha|2019-09-22|>', '<|Fecha|2008-0

**Leer .csv de estaciones**

In [31]:
import os
import csv

# Definir la ruta absoluta del archivo CSV
ruta_absoluta_csv = os.path.abspath('estaciones_distintas.csv')

# Verificar que el archivo existe antes de intentar leerlo
if os.path.exists(ruta_absoluta_csv):
    # Leer el archivo CSV y cargar los datos en una lista
    aditional_special_tokens_array = []
    with open(ruta_absoluta_csv, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            aditional_special_tokens_array.append("<|Estacion|" + row['Estacion'] + "|>")
    print(aditional_special_tokens_array)
else:
    print(f"El archivo {ruta_absoluta_csv} no existe.")


['<|Estacion|Politecnico|>', '<|Estacion|Nazaret Meteo|>', '<|Estacion|Valencia Olivereta|>', '<|Estacion|Pista Silla|>', '<|Estacion|Conselleria Meteo|>', '<|Estacion|Viveros|>', '<|Estacion|Valencia Centro|>', '<|Estacion|Puerto Moll Tras Ponent|>', '<|Estacion|Avenida Francia|>', '<|Estacion|Bulevard Sud|>', '<|Estacion|Puerto llit antic Turia|>', '<|Estacion|Puerto Valencia|>', '<|Estacion|Moli del Sol|>']


In [29]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("flax-community/gpt-2-spanish", bos_token='<bos>', eos_token='<eos>', pad_token='<pad>')
model = AutoModelForCausalLM.from_pretrained("flax-community/gpt-2-spanish")

special_tokens_dict = {
    "additional_special_tokens": aditional_special_tokens_array
}

num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

model.resize_token_embeddings(len(tokenizer))

unk_tok_emb = model.transformer.wte.weight.data[tokenizer.unk_token_id, :]
for i in range(num_added_toks):
    model.transformer.wte.weight.data[-(i+1), :] = unk_tok_emb

# Verificar si los tokens han sido añadidos correctamente
print(tokenizer.additional_special_tokens)
print(tokenizer.convert_tokens_to_ids(tokenizer.additional_special_tokens))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 50272. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


['<|Estacion|Politecnico|>', '<|Estacion|Nazaret Meteo|>', '<|Estacion|Valencia Olivereta|>', '<|Estacion|Pista Silla|>', '<|Estacion|Conselleria Meteo|>', '<|Estacion|Viveros|>', '<|Estacion|Valencia Centro|>', '<|Estacion|Puerto Moll Tras Ponent|>', '<|Estacion|Avenida Francia|>', '<|Estacion|Bulevard Sud|>', '<|Estacion|Puerto llit antic Turia|>', '<|Estacion|Puerto Valencia|>', '<|Estacion|Moli del Sol|>']
[50259, 50260, 50261, 50262, 50263, 50264, 50265, 50266, 50267, 50268, 50269, 50270, 50271]


In [34]:
import torch
from torch.utils.data import Dataset
from tqdm import tqdm
from transformers import AutoTokenizer

class GPT2Dataset(Dataset):

    def __init__(self, tokenizer, archivo_csv='historico_temperaturas.csv', max_length=768):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []

        print('Cargando datos...')
        with open(archivo_csv, 'r') as file:
            data = file.readlines()
        
        print('Cantidad de filas:', len(data))

        for line in tqdm(data, total=len(data)):
            encodings_dict = tokenizer(line, truncation=True, max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

dataset = GPT2Dataset(tokenizer)

train_size = int(0.99 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

train_dataloader = DataLoader(
            train_dataset, 
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )


Cargando datos...
Cantidad de filas: 43371


100%|██████████| 43371/43371 [00:11<00:00, 3658.20it/s]

42,937 training samples
  434 validation samples


In [35]:
# some parameters to train
epochs = 1
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8
# this produces sample output every x steps
sample_every = 500
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

/Users/tomas/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
total_t0 = time.time()

model = model.to(device)

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]
        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    
    t0 = time.time()

    total_eval_loss = 0
    nb_eval_steps = 0

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))